In [1]:
from strokes import StrokePatientsMIDataset
import scipy
from torcheeg.transforms import  Select,BandSignal,Compose,ToTensor
from typing import Callable, Dict, Union, List
import numpy as np
import soxr
from downsample import SetSamplingRate
from baseline import BaselineCorrection
from base_transform import EEGTransform

dataset = StrokePatientsMIDataset(root_path='./subdataset',
                                #  io_path='.torcheeg/datasets_1733374842301_f9t8X',
                        chunk_size=500,  # 1 second
                        overlap = 0,
                        offline_transform=Compose(
                                [BaselineCorrection(),
                                BandSignal(sampling_rate=484,band_dict={'frequency_range':[8,40]})]),
                        online_transform=Compose(
                                [ToTensor()]),
                
                        label_transform=Select('label'),
                        num_worker=8
)
print(dataset[0][0].shape) #EEG shape(1,30,500)
print(dataset[0][1])  # label (int)
print(len(dataset))

/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2025-01-02 14:18:58] INFO (torcheeg/MainThread) 🔍 | Processing EEG data. Processed EEG data has been cached to .torcheeg/datasets_1735798738017_Nm1uq.
[2025-01-02 14:18:58] INFO (torcheeg/MainThread) ⏳ | Monitoring the detailed processing of a record for debugging. The processing of other records will only be reported in percentage to keep it clean.
[PROCESS]: 100%|██████████| 1/1 [00:00<00:00, 81.19it/s]

[RECORD ./subdataset/sourcedata/sub-50/sub-50_task-motor-imagery_eeg.mat]: 0it [00:00, ?it/s]
[RECORD ./subdataset/sourcedata/sub-50/sub-50_task-motor-imagery_eeg.mat]: 1it [00:00,  4.37it/s]
[RECORD ./subdataset/sourcedata/sub-50/sub-50_task-motor-imagery_eeg.mat]: 15it [00:00, 56.52it/s]
[RECORD ./subdataset/sourcedata/s

torch.Size([1, 500, 30])
0
160


In [2]:
HYPERPARAMETERS = {
    "seed": 42,
    "batch_size": 16,
    "lr": 1e-5,
    "weight_decay": 1e-5,
    "num_epochs": 30,
}

In [ ]:
eeg_data = torch.rand(5, 30, 1, 500)  # 示例输入数据
# 参数根据任务调整
patch_size = 4  # Patch的大小
in_chans = 30   # 输入信号通道数
num_classes = 3 # 输出类别数
embed_dim = 96  # 嵌入维度
depths = (2, 2, 6, 2)  # 每层Transformer块的深度
num_heads = (3, 6, 12, 24)  # 每层的注意力头数
window_size = 6  # 窗口大小

model = SwinTransformer(
    patch_size=patch_size,
    in_chans=in_chans,
    num_classes=num_classes,
    embed_dim=embed_dim,
    depths=depths,
    num_heads=num_heads,
    window_size=window_size,
    eeg_channel=in_chans,
    num_eegsample=384
)


In [3]:
from eca import ECA
# from model import SwinTransformer
from model_EEG_selfsupervised import swin_eeg
import torch
import torch.nn as nn

testdata = torch.randn(10, 30, 1, 500)
print(testdata.shape)

class ECASwinTransformerModel(nn.Module):
    def __init__(self, num_channels):
        super(ECASwinTransformerModel, self).__init__()
        self.eca = ECA(input_size=500,num_attention_heads=2,hidden_size=500,hidden_dropout_prob=0.5)
        self.swin_transformer = swin_eeg()

    def forward(self, x):
        # x shape: (batch_size, channels, time)
        x = x.view(x.size(0),30,500)
        x = self.eca(x)
        # Reshape for Swin Transformer
        x = x.view(x.size(0),30,1,500)
        x = self.swin_transformer(x)
        return x

torch.Size([10, 30, 1, 500])


In [4]:
from model_EEG_selfsupervised import swin_eeg

In [5]:
import torch

testmodel = ECASwinTransformerModel(num_channels=30)
testdata = torch.randn(10, 30, 1, 500)
output = testmodel(testdata)
print(output[0])

/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


tensor([-0.5480,  0.2277], grad_fn=<SelectBackward0>)


In [6]:
import os
import shutil

def delete_folder_if_exists(target_folder_name):
    # 获取父文件夹中的所有内容
    parent_folder = os.getcwd()
    for folder_name in os.listdir(parent_folder):
        folder_path = os.path.join(parent_folder, folder_name)

        # 检查是否是文件夹并且名称是否匹配
        if os.path.isdir(folder_path) and folder_name == target_folder_name:
            try:
                # 删除目标文件夹
                shutil.rmtree(folder_path)
                print(f"已删除文件夹: {folder_path}")
            except Exception as e:
                print(f"删除文件夹 {folder_path} 时出错: {e}")




In [7]:
for i, (training_dataset, test_dataset) in enumerate(k_fold.split(dataset)):
    if i==3:
        training_loader = DataLoader(training_dataset,
                                batch_size=HYPERPARAMETERS['batch_size'],
                                shuffle=True)
        test_loader = DataLoader(test_dataset,
                                batch_size=HYPERPARAMETERS['batch_size'],
                                shuffle=False)
        # Assuming test_loader is your DataLoader
        for batch in test_loader:
            # Assuming your DataLoader yields data as (inputs, labels)
            inputs, labels = batch
            print("Inputs shape:", inputs.shape)
            print("Labels shape:", labels.shape)
            break  # To print the shape of only the first batch


NameError: name 'k_fold' is not defined

In [21]:
from torcheeg.model_selection import KFoldPerSubjectGroupbyTrial
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torch.utils.data import DataLoader
# from model import SwinTransformer
# from torcheeg.trainers import ClassifierTrainer
from classifier import ClassifierTrainer

k_fold = KFoldPerSubjectGroupbyTrial(
    n_splits=4,
    shuffle=True,
    # split_path='.torcheeg/model_selection_1733281091381_ZZiR1',
    random_state=42)

training_metrics = []
test_metrics = []

for i, (training_dataset, test_dataset) in enumerate(k_fold.split(dataset)):
    # delete_folder_if_exists(target_folder_name='lightning_logs')
    model = ECASwinTransformerModel(num_channels=30)
    trainer = ClassifierTrainer(model=model,
                                num_classes=2,
                                lr=HYPERPARAMETERS['lr'],
                                weight_decay=HYPERPARAMETERS['weight_decay'],
                                metrics=["accuracy"],
                                accelerator="gpu")
     
    training_loader = DataLoader(training_dataset,
                             batch_size=HYPERPARAMETERS['batch_size'],
                             shuffle=True)
    test_loader = DataLoader(test_dataset,
                             batch_size=HYPERPARAMETERS['batch_size'],
                             shuffle=False)
    # 提前停止回调
    early_stopping_callback = EarlyStopping(
        monitor='train_loss',
        patience=18,  # 提前停止的耐心参数
        mode='min',
        verbose=True
    )
    trainer.fit(training_loader,
                test_loader,
                max_epochs=HYPERPARAMETERS['num_epochs'],
                callbacks=[early_stopping_callback],
                enable_progress_bar=True,
                enable_model_summary=False,
                limit_val_batches=0.0)
    # training_result = trainer.test(training_loader,
    #                                enable_progress_bar=True,
    #                                enable_model_summary=True)[0]
    test_result = trainer.test(test_loader,
                               enable_progress_bar=True,
                               enable_model_summary=True)[0]
    # training_metrics.append(training_result["test_accuracy"])
    test_metrics.append(test_result["test_accuracy"])
     
print({
    # "training_metric_avg": np.mean(training_metrics),
    # "training_metric_std": np.std(training_metrics),
    "test_metric_avg": np.mean(test_metrics),
    "test_metric_std": np.std(test_metrics)
})

[2024-12-06 09:49:24] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2024-12-06 09:49:24] INFO (torcheeg/MainThread) 😊 | Please set split_path to .torcheeg/model_selection_1733449764345_TndCe for the next run, if you want to use the same setting for the experiment.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0: 100%|██████████| 8/8 [00:00<00:00, 15.61it/s, loss=0.811, train_loss=0.851, train_accuracy=0.500]

Metric train_loss improved. New best score: 0.851


Epoch 1: 100%|██████████| 8/8 [00:00<00:00, 18.09it/s, loss=0.718, train_loss=0.596, train_accuracy=0.625]

Metric train_loss improved by 0.255 >= min_delta = 0.0. New best score: 0.596


...

Epoch 79: 100%|██████████| 8/8 [00:00<00:00, 18.93it/s, loss=0.0209, train_loss=0.00637, train_accuracy=1.000]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 78.92it/s]

[2024-12-06 09:56:08] INFO (torcheeg/MainThread) 
[Test] test_loss: 1.192 test_accuracy: 0.775 



Testing DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 68.09it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.7749999761581421
        test_loss           1.1924124956130981
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'test_metric_avg': 0.6562499975164732, 'test_metric_std': 0.10612738329601575}


In [22]:
for i, score in enumerate(test_metrics):
    print(i,score)

0 0.625
1 0.550000011920929
2 0.5249999761581421
3 0.625
4 0.6499999761581421
5 0.675000011920929
6 0.625
7 0.7749999761581421
8 0.8500000238418579
9 0.7250000238418579
10 0.4749999940395355
11 0.7749999761581421


In [23]:

# 每4个计算平均值
averages = [sum(test_metrics[i:i+4]) / len(test_metrics[i:i+4]) for i in range(0, len(test_metrics), 4)]

for i, ave in enumerate(averages):
    # if ave>0.6:
        print(f"Subject{i+1}: {ave:.3f}")
# print("每4个数的平均值:", averages)

Subject1: 0.581
Subject2: 0.681
Subject3: 0.706


In [33]:
import numpy as np
print({
    # "training_metric_avg": np.mean(training_metrics),
    # "training_metric_std": np.std(training_metrics),
    "len": len(test_metrics),
    "test_metric_avg": np.mean(test_metrics),
    "test_metric_std": np.std(test_metrics)
})

{'len': 8, 'test_metric_avg': 0.5406249985098839, 'test_metric_std': 0.08654614558509252}
